In [1]:
import pyodbc
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
import datetime as dt
import json
from scipy import stats
import numpy as np
import sys
import math
from scipy.stats.stats import pearsonr

In [6]:
df_Merged = pd.read_csv('pre_pro_EFtot.csv')

expYS_list = df_Merged['Expected_years_of_schooling'].tolist()
CPI_list = df_Merged['CPI2015'].tolist()
EF_list = df_Merged['EFtotal'].tolist()
crop_land = df_Merged['crop_land'].tolist()
grazing_land = df_Merged['grazing_land'].tolist()
forest_land = df_Merged['forest_land'].tolist()
fishing_ground = df_Merged['fishing_ground'].tolist()
built_up_land = df_Merged['built_up_land'].tolist()
carbon = df_Merged['carbon'].tolist()
GDP = df_Merged['GDP'].tolist()
HDI = df_Merged['Human_Development_Index'].tolist()
GNI_list_bad = df_Merged['Gross_national_income_GNI_per_capita'].tolist()
GINI_list_bad = df_Merged['GINI'].tolist()
GINI_list = []
GNI_list = []

for i in GNI_list_bad:
    intValue = int(i)
    GNI_list.append(intValue)

GINI_list1 = list(map(lambda x: 0.0 if math.isnan(x) else x, GINI_list_bad))
avgGINI = sum(GINI_list1) / len(GINI_list1)    
GINI_list = list(map(lambda x: avgGINI if math.isnan(x) else x, GINI_list_bad))

slope, intercept, r_value, p_value, std_err = stats.linregress(x=GDP,y=CPI_list)

# This calculates the predicted value for each observed value
obs_values = CPI_list
pred_values = []
for i in GDP:
    pred_i = float(i) * float(slope) + float(intercept)
    pred_values.append(pred_i)

# This prints the residual for each pair of observations
residual = []
for obs_v,pred_v in zip(obs_values,pred_values):
    res_v = obs_v - pred_v
    residual.append(res_v)
    
res_CPI = residual

del df_Merged['Unnamed: 0']
res_CPI = pd.Series(res_CPI)
df_Merged['residual_CPI'] = res_CPI.values
df_corr = df_Merged.corr()

list_of_lists = (
    GDP,
    HDI,
    expYS_list,
    CPI_list,
    EF_list,
    crop_land,
    grazing_land,
    forest_land,
    fishing_ground,
    built_up_land,
    carbon,
    GINI_list,
    GNI_list
)

list_of_names = (
    'GDP',
    'HDI',
    'expYS_list',
    'CPI_list',
    'EF_list',
    'crop_land',
    'grazing_land',
    'forest_land',
    'fishing_ground',
    'built_up_land',
    'carbon',
    'GINI_list',
    'GNI_list'
)

result = []

for i,j in zip(list_of_lists, list_of_names):
    result.append(pearsonr(res_CPI, i))

corr_coef, p_value = zip(*result)

corr_coef = list(corr_coef)
p_value = list(p_value)
list_of_names = list(list_of_names)

dict_df = {'Indicator': list_of_names, 'Correlation Coefficient': corr_coef, 'P Value': p_value}
df_pearson = pd.DataFrame(dict_df)

#df_pearson.to_csv("pearson.csv")
#df_corr.to_csv("correlation.csv")

In [16]:
df = df_Merged

df = df[np.abs(df['crop_land']-df['crop_land'].mean()) <= (2*df['crop_land'].std())]
crop_land = df['crop_land'].tolist()

trace1 = go.Scatter(
            x = CPI_list,
            y = GINI_list,
            mode = 'markers'
            )

data = [trace1]

layout = dict(xaxis = dict(title = 'EF of crop land (GHA)',
                          #type='log',
                          #autorange=True
                          ),
              yaxis = dict(title = 'Expected Year of Schooling',
                          #type='log',
                          #autorange=True
                          ),
             )

fig1 = dict(data=data, layout=layout)
py.iplot(fig1, filename='styled-line')

In [5]:
df = pd.read_csv('OGP_Explorer__selected_Commitments_irm_data.csv')
df['Country name'].nunique()

93